# Training the SepHeads model

__Objective:__ train the SepHeads model (pre-trained DeBERTa text encoder, annotator-specific classification heads).

In [1]:
import sys
from copy import deepcopy
import pandas as pd
import torch
import datasets
import transformers

sys.path.append('../modules/')

from custom_logger import get_logger
from data_utils import subsample_dataset
from model_utils import get_deberta_model
from models import DebertaWithAnnotatorHeads
from training_metrics import compute_metrics_sklearn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger = get_logger('sepheads_model_training')

%load_ext autoreload
%autoreload 2

/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data.
DATASET_NAME = 'kumar'

DATASET_PATHS = {
    'popquorn': '../data/samples/POPQUORN_offensiveness.csv',
    'kumar': {
        'train': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_train.csv',
        # 'train':  '/data/milanlp/moscato/personal_hate_bounds_data/kumar_processed_with_ID_and_full_perspective_clean.csv',
        'test': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_test.csv',
        'annotators_data': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/annotators_data.csv'
    }
}

logger.info(f'Loading data from: {DATASET_PATHS[DATASET_NAME]}')

training_data = pd.read_csv(DATASET_PATHS[DATASET_NAME]['train']).drop(columns=['extreme_annotator'])
test_data = pd.read_csv(DATASET_PATHS[DATASET_NAME]['test']).drop(columns=['extreme_annotator'])

2025-03-04 09:58:29,363 - sepheads_model_training - INFO - Loading data from: {'train': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_train.csv', 'test': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_test.csv', 'annotators_data': '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/annotators_data.csv'}


In [3]:
N_ANNOTATORS_TEST = None
OPTIMAL_N_TRAINING_DATAPOINTS = 10000

if N_ANNOTATORS_TEST is not None:
    logger.warning(f'Testing with {N_ANNOTATORS_TEST} annotators')

    annotator_ids = sorted(training_data['annotator_id'].unique())[:N_ANNOTATORS_TEST]

    training_data = training_data[training_data['annotator_id'].isin(annotator_ids)].reset_index(drop=True)
    test_data = test_data[test_data['annotator_id'].isin(annotator_ids)].reset_index(drop=True)
else:
    if OPTIMAL_N_TRAINING_DATAPOINTS is not None:
        logger.info(
            f'Subsampling data to {OPTIMAL_N_TRAINING_DATAPOINTS}'
            ' training datapoints'
        )

        training_data, test_data = subsample_dataset(
            training_data,
            test_data,
            OPTIMAL_N_TRAINING_DATAPOINTS,
            DATASET_PATHS[DATASET_NAME]['annotators_data']
        )

    annotator_ids = sorted(training_data['annotator_id'].unique())

logger.info(
    f'N annotators: {len(annotator_ids)} | N training samples: {len(training_data)}'
    f' | N test samples: {len(test_data)}'
)

2025-03-04 09:58:30,598 - sepheads_model_training - INFO - Subsampling data to 10000 training datapoints
2025-03-04 09:58:30,620 - sepheads_model_training - INFO - Optimal N datapoints: 10000 | Optimal N annotators: 81
2025-03-04 09:58:30,621 - sepheads_model_training - INFO - Subsampling the data (manually including all extreme annotators)
2025-03-04 09:58:30,635 - sepheads_model_training - INFO - N training datapoints: 12658 | N annotators: 236
2025-03-04 09:58:30,645 - sepheads_model_training - INFO - N annotators: 236 | N training samples: 12658 | N test samples: 3028


In [5]:
# Instantiate the DeBERTa text encoder.
logger.info('Instantiating the SepHeads model')

num_labels = training_data['toxic_score'].unique().shape[0]

model_dir = '/data1/shared_models/'

logger.info(f'N labels found in training data: {num_labels}')

deberta_tokenizer, deberta_model = get_deberta_model(
    num_labels,
    model_dir,
    device,
    use_custom_head=False,
    pooler_out_features=768,  # Default: 768.
    pooler_drop_prob=0.0,  # Default: 0.0
    classifier_drop_prob=0.1,  # Default: 0.1
    use_fast_tokenizer=False
)

deberta_with_annotator_heads_model = DebertaWithAnnotatorHeads(
    deberta_encoder=deepcopy(deberta_model.deberta),
    deberta_pooler=deepcopy(deberta_model.pooler),
    deberta_dropout=deepcopy(deberta_model.dropout),
    num_labels=num_labels,
    annotator_ids=annotator_ids,
)

del deberta_model

2025-03-04 09:59:15,616 - sepheads_model_training - INFO - Instantiating the SepHeads model
2025-03-04 09:59:15,618 - sepheads_model_training - INFO - N labels found in training data: 2
2025-03-04 09:59:15,618 - sepheads_model_training - INFO - Instantiating DeBERTa tokenizer
2025-03-04 09:59:16,110 - sepheads_model_training - INFO - Instantiating DeBERTa model with default classification head
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize_function(examples):
    return deberta_tokenizer(
        examples["text"],
        padding='max_length',
        truncation=True,
        max_length=512,
        # return_tensors='pt'
    )

In [6]:
# Create tokenized datasets.
logger.info('Creating tokenized datasets')

tokenized_training_data = (
    # Create datast object from the DataFrame.
    datasets.Dataset.from_dict(
        training_data[[
            'comment',
            'toxic_score',
            'annotator_id'
        ]].rename(
            columns={
                'comment': 'text',
                'toxic_score': 'label',
                'annotator_id': 'annotator_ids',
            }
        )
        .to_dict(orient='list')
    )
    # Tokenize.
    .map(tokenize_function, batched=True)
    # Remove useless column.
    .remove_columns("text")
    .shuffle()
    .flatten_indices()
)

tokenized_test_data = (
    # Create datast object from the DataFrame.
    datasets.Dataset.from_dict(
        test_data[[
            'comment',
            'toxic_score',
            'annotator_id'
        ]].rename(
            columns={
                'comment': 'text',
                'toxic_score': 'label',
                'annotator_id': 'annotator_ids',
            }
        )
        .to_dict(orient='list')
    )
    # Tokenize.
    .map(tokenize_function, batched=True)
    # Remove useless column.
    .remove_columns("text")
)

2025-02-19 15:36:18,857 - sepheads_model_training - INFO - Creating tokenized datasets
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2904/2904 [00:00<00:00, 3135.43 examples/s]


In [7]:
EXPERIMENT_ID = 'sepheads_model_training_test'
MODEL_OUTPUT_DIR = f'/data1/moscato/personalised-hate-boundaries-data/models/{EXPERIMENT_ID}/'
N_EPOCHS = 5

training_args = transformers.TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",  # Options: 'no', 'epoch', 'steps' (requires the `save_steps` argument to be set though).
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=5e-6,
    per_device_train_batch_size=16,  # Default: 8.
    gradient_accumulation_steps=1,  # Default: 1.
    per_device_eval_batch_size=32,  # Default: 8.
    num_train_epochs=N_EPOCHS,
    warmup_ratio=0.0,  # For linear warmup of learning rate.
    metric_for_best_model="f1",
    push_to_hub=False,
    # label_names=list(roberta_classifier.config.id2label.keys()),
    logging_strategy='epoch',
    logging_first_step=True,
    logging_dir=f'../tensorboard_logs/{EXPERIMENT_ID}/',
    # logging_steps=10,
    disable_tqdm=False
)

data_collator = transformers.DataCollatorWithPadding(tokenizer=deberta_tokenizer)

trainer = transformers.Trainer(
    model=deberta_with_annotator_heads_model,
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_test_data,
    data_collator=data_collator,
    tokenizer=deberta_tokenizer,
    compute_metrics=compute_metrics_sklearn,
)

/tmp/ipykernel_416901/3345286892.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [8]:
training_output = trainer.train()

/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.682400,0.662616,0.608471,0.577944,0.581421,0.577200
2,0.651000,0.643827,0.633953,0.602990,0.608768,0.601591
3,0.633300,0.635695,0.635331,0.602119,0.609538,0.600776
4,0.623000,0.632099,0.635675,0.601713,0.609698,0.600411
5,0.616100,0.630548,0.637052,0.603566,0.611381,0.602184


/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/moscato/miniconda3/envs/phb/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/mo